In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
data=pd.read_csv("Scenario-A-merged_5s.csv")
data=data.replace([np.inf,-np.inf],np.nan)
data=data.dropna()
data.head()

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,...,Bwd IAT Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,10.0.2.15,53913,216.58.208.46,80,6,435,0.0,4597.701149,435.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
1,10.0.2.15,53913,216.58.208.46,80,6,259,0.0,7722.007722,259.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
2,10.0.2.15,53913,216.58.208.46,80,6,891,0.0,2244.668911,891.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
3,10.0.2.15,53913,216.58.208.46,80,6,1074,0.0,1862.197393,1074.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
4,10.0.2.15,53913,216.58.208.46,80,6,315,0.0,6349.206349,315.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84187 entries, 0 to 84186
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Source IP          84187 non-null  object 
 1    Source Port       84187 non-null  int64  
 2    Destination IP    84187 non-null  object 
 3    Destination Port  84187 non-null  int64  
 4    Protocol          84187 non-null  int64  
 5    Flow Duration     84187 non-null  int64  
 6    Flow Bytes/s      84187 non-null  float64
 7    Flow Packets/s    84187 non-null  float64
 8    Flow IAT Mean     84187 non-null  float64
 9    Flow IAT Std      84187 non-null  float64
 10   Flow IAT Max      84187 non-null  int64  
 11   Flow IAT Min      84187 non-null  int64  
 12  Fwd IAT Mean       84187 non-null  float64
 13   Fwd IAT Std       84187 non-null  float64
 14   Fwd IAT Max       84187 non-null  int64  
 15   Fwd IAT Min       84187 non-null  int64  
 16  Bwd IAT Mean       84187 no

In [3]:
data['Source IP'].value_counts()

Source IP
131.202.240.150    10472
10.0.2.15          10204
10.8.8.130          6311
10.8.0.14           6294
131.202.240.242     5975
                   ...  
173.194.61.240         1
62.41.83.228           1
108.162.232.203        1
62.75.252.35           1
74.125.228.232         1
Name: count, Length: 766, dtype: int64

由于IP特征非常之稀疏，我不建议在起步阶段纳入特征范围内。但是后续如果想搞图神经网络倒是可以考虑引入一下。现在把这两个删掉。

In [4]:
data=data.drop(['Source IP', ' Destination IP', ' Source Port', ' Destination Port'],axis=1)
data.head()

,Protocol,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Mean,Fwd IAT Std,...,Bwd IAT Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,6,435,0.0,4597.701149,435.0,0.0,435,435,0.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
1,6,259,0.0,7722.007722,259.0,0.0,259,259,0.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
2,6,891,0.0,2244.668911,891.0,0.0,891,891,0.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
3,6,1074,0.0,1862.197393,1074.0,0.0,1074,1074,0.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
4,6,315,0.0,6349.206349,315.0,0.0,315,315,0.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR


In [5]:
data[' Protocol'].value_counts()

 Protocol
17    43695
6     40492
Name: count, dtype: int64

In [6]:
X=data.iloc[:,:-1]
y=data.iloc[:,-1]
mms=MinMaxScaler()
le=LabelEncoder()
X_scaled=mms.fit_transform(X)
y_scaled=le.fit_transform(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(67349, 24) (16838, 24) (67349,) (16838,)


In [8]:
clf=XGBClassifier()
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      2986
           1       1.00      1.00      1.00     13852

    accuracy                           0.99     16838
   macro avg       0.99      0.99      0.99     16838
weighted avg       0.99      0.99      0.99     16838



In [9]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,y_pred)


0.9881683876958581

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

models={'lr':LogisticRegression(),'svm':SVC(),'dt':DecisionTreeClassifier(),'etc':ExtraTreesClassifier(),'rf':RandomForestClassifier(),'adb':AdaBoostClassifier(),'gbdt':GradientBoostingClassifier(),'gnb':GaussianNB(),'mnb':MultinomialNB(),'xgb':XGBClassifier()}
for name in models.keys():
    clf=models[name]
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    print('===================={}====================='.format(name))
    print("Accuracy:%.4f"%(accuracy_score(y_test,y_pred)))
    print("F1:%.4f"%(f1_score(y_test,y_pred)))
    print("Precision:%.4f"%(precision_score(y_test,y_pred)))
    print("Recall:%.4f"%(recall_score(y_test,y_pred)))
    print("AUC:%.4f"%(roc_auc_score(y_test,y_pred)))
    print('\n')

e:\anaconda\envs\kan\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


====================lr=====================
Accuracy:0.9161
F1:0.9495
Precision:0.9409
Recall:0.9582
AUC:0.8394


====================svm=====================
Accuracy:0.9286
F1:0.9572
Precision:0.9427
Recall:0.9722
AUC:0.8491


====================dt=====================
Accuracy:0.9816
F1:0.9888
Precision:0.9894
Recall:0.9882
AUC:0.9697


====================etc=====================
Accuracy:0.9874
F1:0.9923
Precision:0.9909
Recall:0.9938
AUC:0.9756


====================rf=====================
Accuracy:0.9903
F1:0.9941
Precision:0.9922
Recall:0.9961
AUC:0.9798


====================adb=====================
Accuracy:0.9366
F1:0.9619
Precision:0.9510
Recall:0.9731
AUC:0.8702


====================gbdt=====================
Accuracy:0.9635
F1:0.9780
Precision:0.9691
Recall:0.9871
AUC:0.9205


====================gnb=====================
Accuracy:0.8060
F1:0.8673
Precision:0.9917
Recall:0.7706
AUC:0.8704


====================mnb=====================
Accuracy:0.8116
F1:0.8951
Precision:0

In [12]:
ohe=OneHotEncoder(sparse=False)
ohe.fit_transform(pd.DataFrame(y_test))

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [ ]:
import torch  
import torch.nn as nn  
import torch.optim as optim  
from torch.utils.data import DataLoader, TensorDataset, random_split  
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score
  
# 假设你的数据已经以PyTorch.tensor的形式准备好了  
# 输入数据 X 和标签 Y  
# X.shape = (num_samples, 24)  
# Y.shape = (num_samples, 2)  
# 这里只是示例，请用你的实际数据替换  
  
# 示例数据  
num_samples = 1000  
input_dim = 24  
num_classes = 8 

loaded_dataset = torch.load('dataset_new.pth')
train_dataset = TensorDataset(loaded_dataset['train_input'],loaded_dataset['train_label'])
test_dataset = TensorDataset(loaded_dataset['test_input'],loaded_dataset['test_label'])
  
# 创建DataLoader  
batch_size = 32  
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)  
  
class DNN(nn.Module):
    def __init__(self, input_dim=24, hidden_dim1=128, hidden_dim2=64, output_dim=8):
        super(DNN, self).__init__()
        # 第一个隐藏层
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        # 第二个隐藏层
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        # 输出层
        self.fc3 = nn.Linear(hidden_dim2, output_dim)

        self.activate1=nn.ReLU()
        self.activate2=nn.ReLU()
        self.activate3=nn.Softmax()

    def forward(self, x):
        # 输入x通过第一个隐藏层
        x = self.activate1(self.fc1(x))
        # 第一个隐藏层的输出通过第二个隐藏层
        x = self.activate2(self.fc2(x))
        # 第二个隐藏层的输出通过输出层
        x = self.activate3(self.fc3(x))
        return x  
  
# 初始化模型、损失函数和优化器  
model = DNN(input_dim, output_dim=num_classes)  
criterion = nn.BCELoss()  # 二分类交叉熵损失，适用于独热编码  
optimizer = optim.Adam(model.parameters(), lr=0.001)  
  
# 训练模型  
num_epochs = 50
for epoch in range(num_epochs):  
    model.train()  
    running_loss = 0.0  
    for inputs, labels in train_loader:  
        optimizer.zero_grad()  
        outputs = model(inputs)  
        loss = criterion(outputs, labels)  
        loss.backward()  
        optimizer.step()  
        running_loss += loss.item()  
      
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')  
  
# 测试模型  
model.eval()  
correct = 0  
total = 0  
all_labels=[]
all_preds=[]
outputs_prob=[]
with torch.no_grad():  
    for inputs, labels in test_loader:  
        outputs = model(inputs)  
        outputs_prob.extend(outputs.numpy().max(axis=1))
        _, predicted = torch.max(outputs.data, 1)  
        all_preds.extend(predicted.numpy())
        all_labels.extend(labels.numpy().argmax(axis=1))
  
# 计算准确率
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy:.4f}')

# 计算F1分数
f1 = f1_score(all_labels, all_preds, average='macro')
print(f'F1 Score: {f1:.4f}')

# 计算查准率
precision = precision_score(all_labels, all_preds, average='macro')
print(f'Precision: {precision:.4f}')

# 计算召回率
recall = recall_score(all_labels, all_preds, average='macro')
print(f'Recall: {recall:.4f}')

# 如果你的标签是二分类并且输出是概率，可以计算AUC
# 假设outputs_prob是模型输出的概率，需要将其转换为numpy数组
# outputs_prob = outputs[:, 1].cpu().numpy()  # 假设是二分类问题，取第二列的概率
# labels = (labels == 1).astype(int)  # 将标签转换为二进制标签
auc = roc_auc_score(all_labels, outputs_prob)
print(f'AUC: {auc:.4f}')

cpu
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 1.82e-01 | test_loss: 1.84e-01 | reg: 7.43e+01 | : 100%|█| 50/50 [25:42<00:00, 30.85s/


saving model version 0.1
0.9548991322517395 0.9539731740951538
